# am

In [ ]:
// // test

open testing

prototype append t : t -> t -> t

()



In [ ]:
open rust_operators

()



## append

In [ ]:
instance append a dim {int; number} t =
    am.append

()



## /@

In [ ]:
inl (/@) a b =
    b |> append a

()



In [ ]:
// // test

a ;[ "a"; "b" ] /@ a ;[ "c"; "d" ]
|> _equal (a ;[ "a"; "b"; "c"; "d" ] : _ i32 _)

type Mut0 = {mutable l0 : int32}
let rec method1 (v0 : (string [])) : (string []) =
    v0
and method2 (v0 : int32, v1 : Mut0) : bool =
    let v2 : int32 = v1.l0
    let v3 : bool = v2 < v0
    v3
and method3 (v0 : (string []), v1 : (string []), v2 : int32) : bool =
    let v3 : int32 = v0.Length
    let v4 : bool = v2 < v3
    if v4 then
        let v5 : string = v0.[int v2]
        let v6 : string = v1.[int v2]
        let v7 : bool = v5 = v6
        if v7 then
            let v8 : int32 = v2 + 1
            method3(v0, v1, v8)
        else
            false
    else
        true
and method0 () : unit =
    let v0 : string = "a"
    let v1 : string = "b"
    let v2 : (string []) = [|v0; v1|]
    let v3 : (string []) = method1(v2)
    let v4 : string = "c"
    let v5 : string = "d"
    let v6 : (string []) = [|v4; v5|]
    let v7 : (string []) = method1(v6)
    let v8 : int32 = v3.Length
    let v9 : int32 = v7.Length
    let v10 : int32 = v8 + v9
    let v11 : (string []) = Array.ze

## collect

In [ ]:
inl collect forall t r. (fn : t -> a i32 r) (items : a i32 t) : a i32 r =
    items
    |> am.map fn
    |> am.fold (/@) (a ;[])

()



## choose

In [ ]:
inl choose f l =
    (l, [])
    ||> am.foldBack fun x acc =>
        match f x with
        | Some y => y :: acc
        | None => acc
    |> listm.toArray

()



In [ ]:
// // test

(am.init 10i32 id : a _ _)
|> choose (fun x => if x % 2 = 0 then Some x else None)
|> _equal (a ;[ 0; 2; 4; 6; 8 ] : _ i32 _)

type Mut0 = {mutable l0 : int32}
and UH0 =
    | UH0_0 of int32 * UH0
    | UH0_1
and Mut1 = {mutable l0 : int32; mutable l1 : UH0}
and [<Struct>] US0 =
    | US0_0
    | US0_1 of f1_0 : int32
let rec method1 (v0 : Mut0) : bool =
    let v1 : int32 = v0.l0
    let v2 : bool = v1 < 10
    v2
and method2 (v0 : int32, v1 : Mut1) : bool =
    let v2 : int32 = v1.l0
    let v3 : bool = v2 < v0
    v3
and method4 (v0 : UH0, v1 : int32) : int32 =
    match v0 with
    | UH0_0(v2, v3) -> (* Cons *)
        let v4 : int32 = v1 + 1
        method4(v3, v4)
    | UH0_1 -> (* Nil *)
        v1
and method5 (v0 : (int32 []), v1 : UH0, v2 : int32) : int32 =
    match v1 with
    | UH0_0(v3, v4) -> (* Cons *)
        v0.[int v2] <- v3
        let v5 : int32 = v2 + 1
        method5(v0, v4, v5)
    | UH0_1 -> (* Nil *)
        v2
and method3 (v0 : UH0) : (int32 []) =
    let v1 : int32 = 0
    let v2 : int32 = method4(v0, v1)
    let v3 : (int32 []) = Array.zeroCreate<int32> (v2)
    let v4 : int32 = 0


## sum

In [ ]:
inl sum (a' : a _ _) =
    a' |> am.fold (+) 0

()



In [ ]:
// // test

am.init 10i32 id
|> sum
|> _equal 45

type Mut0 = {mutable l0 : int32}
and Mut1 = {mutable l0 : int32; mutable l1 : int32}
let rec method1 (v0 : Mut0) : bool =
    let v1 : int32 = v0.l0
    let v2 : bool = v1 < 10
    v2
and method2 (v0 : int32, v1 : Mut1) : bool =
    let v2 : int32 = v1.l0
    let v3 : bool = v2 < v0
    v3
and method0 () : unit =
    let v0 : (int32 []) = Array.zeroCreate<int32> (10)
    let v1 : Mut0 = {l0 = 0} : Mut0
    while method1(v1) do
        let v3 : int32 = v1.l0
        v0.[int v3] <- v3
        let v4 : int32 = v3 + 1
        v1.l0 <- v4
        ()
    let v5 : int32 = v0.Length
    let v6 : Mut1 = {l0 = 0; l1 = 0} : Mut1
    while method2(v5, v6) do
        let v8 : int32 = v6.l0
        let v9 : int32 = v6.l1
        let v10 : int32 = v0.[int v8]
        let v11 : int32 = v9 + v10
        let v12 : int32 = v8 + 1
        v6.l0 <- v12
        v6.l1 <- v11
        ()
    let v13 : int32 = v6.l1
    let v14 : string = $"__expect / actual: %A{v13} / expected: %A{45}"
    let v15 : bool = v13

## init_series

In [ ]:
inl init_series start end inc =
    inl total = conv ((end - start) / inc) + 1
    am.init total (conv >> (*) inc >> (+) start) : a i32 _

()



In [ ]:
// // test

init_series 0 1 0.5
|> _equal (a ;[0f64; 0.5; 1])

type Mut0 = {mutable l0 : int32}
let rec method1 (v0 : Mut0) : bool =
    let v1 : int32 = v0.l0
    let v2 : bool = v1 < 3
    v2
and method2 (v0 : (float [])) : (float []) =
    v0
and method3 (v0 : (float []), v1 : (float []), v2 : int32) : bool =
    let v3 : int32 = v0.Length
    let v4 : bool = v2 < v3
    if v4 then
        let v5 : float = v0.[int v2]
        let v6 : float = v1.[int v2]
        let v7 : bool = v5 = v6
        if v7 then
            let v8 : int32 = v2 + 1
            method3(v0, v1, v8)
        else
            false
    else
        true
and method0 () : unit =
    let v0 : (float []) = Array.zeroCreate<float> (3)
    let v1 : Mut0 = {l0 = 0} : Mut0
    while method1(v1) do
        let v3 : int32 = v1.l0
        let v4 : float = float v3
        let v5 : float = 0.5 * v4
        v0.[int v3] <- v5
        let v6 : int32 = v3 + 1
        v1.l0 <- v6
        ()
    let v7 : (float []) = [|0.0; 0.5; 1.0|]
    let v8 : (float []) = method2(v7)
    let v9 : string 

## head

In [ ]:
inl head (ar : a _ _) =
    if var_is ar || length ar > 0
    then index ar 0
    else error_type "The length of the array should be greater than 0."

()



## last

In [ ]:
inl last (ar : a _ _) =
    inl len = length ar
    if var_is ar || len > 0
    then index ar (len - 1)
    else error_type "The length of the array should be greater than 0."

()



In [ ]:
// // test

am.init 10i32 id
|> last
|> _equal 9

type Mut0 = {mutable l0 : int32}
let rec method1 (v0 : Mut0) : bool =
    let v1 : int32 = v0.l0
    let v2 : bool = v1 < 10
    v2
and method0 () : unit =
    let v0 : (int32 []) = Array.zeroCreate<int32> (10)
    let v1 : Mut0 = {l0 = 0} : Mut0
    while method1(v1) do
        let v3 : int32 = v1.l0
        v0.[int v3] <- v3
        let v4 : int32 = v3 + 1
        v1.l0 <- v4
        ()
    let v5 : int32 = v0.Length
    let v6 : int32 = v5 - 1
    let v7 : int32 = v0.[int v6]
    let v8 : string = $"__expect / actual: %A{v7} / expected: %A{9}"
    let v9 : bool = v7 = 9
    let v10 : bool = v9 = false
    if v10 then
        failwith<unit> v8
method0()



## try_pick

In [ ]:
inl try_pick forall t u. (fn : t -> option u) (array : a _ t) : option u =
    (array, None)
    ||> am.foldBack fun x acc =>
        match acc with
        | Some _ => acc
        | None => x |> fn

()



In [ ]:
// // test

am.init 10i32 id
|> try_pick (fun x => if x = 5i32 then Some x else None)
|> _equal (Some 5i32)

type Mut0 = {mutable l0 : int32}
and [<Struct>] US0 =
    | US0_0
    | US0_1 of f1_0 : int32
and Mut1 = {mutable l0 : int32; mutable l1 : US0}
let rec method1 (v0 : Mut0) : bool =
    let v1 : int32 = v0.l0
    let v2 : bool = v1 < 10
    v2
and method2 (v0 : int32, v1 : Mut1) : bool =
    let v2 : int32 = v1.l0
    let v3 : bool = v2 < v0
    v3
and method0 () : unit =
    let v0 : (int32 []) = Array.zeroCreate<int32> (10)
    let v1 : Mut0 = {l0 = 0} : Mut0
    while method1(v1) do
        let v3 : int32 = v1.l0
        v0.[int v3] <- v3
        let v4 : int32 = v3 + 1
        v1.l0 <- v4
        ()
    let v5 : int32 = v0.Length
    let v6 : US0 = US0_0
    let v7 : Mut1 = {l0 = 0; l1 = v6} : Mut1
    while method2(v5, v7) do
        let v9 : int32 = v7.l0
        let v10 : int32 =  -v9
        let v11 : int32 = v10 + v5
        let v12 : int32 = v11 - 1
        let v13 : US0 = v7.l1
        let v14 : int32 = v0.[int v12]
        let v21 : US0 =
            match v13 with
         

## from_vec

In [ ]:
inl from_vec forall dim el. (vec : rust.vec el) : a dim el =
    inl vec = join vec
    !\($'"fable_library_rust::NativeArray_::array_from(!vec)"')

()



## to_vec

In [ ]:
inl to_vec forall t. (ar : array_base t) : rust.vec t =
    inl ar = join ar
    !\($'"!ar.to_vec()"')

()



## map_vec

In [ ]:
inl map_vec forall dim t u. (fn : t -> u) (ar : a dim t) : a dim u =
    inl fn = join fn
    inl (a ar) = ar
    inl ar = ar |> to_vec
    !\($'"!ar.iter().map(|x| !fn(x.clone())).collect()"')
    |> from_vec

()



## map_base

In [ ]:
inl map_base forall t u. (fn : t -> u) (x : array_base t) : array_base u =
    a x
    |> am.map fn
    |> fun (a x : _ i64 _) => x

()



In [ ]:
inl average forall el {number}. (array : a _ el) : el =
    $"!array |> Array.average"

inl length forall dim el {number}. (array : a dim el) : dim =
    $"!array |> Array.length"

inl parallel_map forall dim el el'. (fn : el -> el') (array : a dim el) : a dim el' =
    $"!array |> Array.Parallel.map !fn"

inl sort_by forall dim el. (fn : el -> _) (array : a dim el) : a dim el =
    $"!array |> Array.sortBy !fn"

inl sort_descending forall dim el. (array : a dim el) : a dim el =
    $"!array |> Array.sortDescending"

inl transpose forall dim el. (array : a dim (a dim el)) : a dim (a dim el) =
    $"!array |> Array.transpose"

inl try_item forall dim el. (i : i32) (array : a dim el) : option el =
    $"!array |> Array.tryItem !i" |> optionm'.unbox

()



## indexed

In [ ]:
inl indexed (array : a _ _) =
    (([], 0), array)
    ||> am.fold fun (acc, i) x =>
        (i, x) :: acc, i + 1
    |> fst
    |> listm.rev
    |> listm.toArray

()



In [ ]:
// // test

am.init 3i32 ((*) 2)
|> indexed
|> _equal (a ;[0i32, 0; 1, 2; 2, 4] : a i32 _)

type Mut0 = {mutable l0 : int32}
and UH0 =
    | UH0_0 of int32 * int32 * UH0
    | UH0_1
and Mut1 = {mutable l0 : int32; mutable l1 : UH0; mutable l2 : int32}
let rec method1 (v0 : Mut0) : bool =
    let v1 : int32 = v0.l0
    let v2 : bool = v1 < 3
    v2
and method2 (v0 : int32, v1 : Mut1) : bool =
    let v2 : int32 = v1.l0
    let v3 : bool = v2 < v0
    v3
and method3 (v0 : UH0, v1 : UH0) : UH0 =
    match v0 with
    | UH0_0(v2, v3, v4) -> (* Cons *)
        let v5 : UH0 = UH0_0(v2, v3, v1)
        method3(v4, v5)
    | UH0_1 -> (* Nil *)
        v1
and method5 (v0 : UH0, v1 : int32) : int32 =
    match v0 with
    | UH0_0(v2, v3, v4) -> (* Cons *)
        let v5 : int32 = v1 + 1
        method5(v4, v5)
    | UH0_1 -> (* Nil *)
        v1
and method6 (v0 : (struct (int32 * int32) []), v1 : UH0, v2 : int32) : int32 =
    match v1 with
    | UH0_0(v3, v4, v5) -> (* Cons *)
        v0.[int v2] <- struct (v3, v4)
        let v6 : int32 = v2 + 1
        method6(v0, v5, v6)
    | UH0_

## enumerate

In [ ]:
inl enumerate forall dim {int; number} t. (ar : a dim t) : a dim (unativeint * t) =
    inl (a ar) = ar
    inl ar = ar |> to_vec
    !\($'"!ar.into_iter().enumerate().map(std::rc::Rc::new).collect()"')
    |> from_vec
    |> map_vec from_pair

()



## indexed'

In [ ]:
inl indexed' forall t dim {int; number} dim' {int; number} u {number}. (ar : a dim t) : a dim' (u * t) =
    ((0, a ;[]), ar)
    ||> am.fold fun (i, acc) x =>
        i + 1, acc /@ a ;[i, x]
    |> snd

()



In [ ]:
// // test

am.init 3i32 ((*) 2)
|> indexed'
|> _equal (a ;[0i32, 0; 1, 2; 2, 4] : a i32 _)

type Mut0 = {mutable l0 : int32}
and Mut1 = {mutable l0 : int32; mutable l1 : int32; mutable l2 : (struct (int32 * int32) [])}
let rec method1 (v0 : Mut0) : bool =
    let v1 : int32 = v0.l0
    let v2 : bool = v1 < 3
    v2
and method2 (v0 : (struct (int32 * int32) [])) : (struct (int32 * int32) []) =
    v0
and method3 (v0 : int32, v1 : Mut1) : bool =
    let v2 : int32 = v1.l0
    let v3 : bool = v2 < v0
    v3
and method4 (v0 : int32, v1 : Mut0) : bool =
    let v2 : int32 = v1.l0
    let v3 : bool = v2 < v0
    v3
and method5 (v0 : (struct (int32 * int32) []), v1 : (struct (int32 * int32) []), v2 : int32) : bool =
    let v3 : int32 = v0.Length
    let v4 : bool = v2 < v3
    if v4 then
        let struct (v5 : int32, v6 : int32) = v0.[int v2]
        let struct (v7 : int32, v8 : int32) = v1.[int v2]
        let v9 : bool = v5 = v7
        let v11 : bool =
            if v9 then
                let v10 : bool = v6 = v8
                v10
            else
                false
   